# ............................................................................................................................................

# Welcome to the RAAIM

Use shift + enter to run a code block or move to the next text block.

Run the next block of code to intialize the program.

* If you run into a "C:\Users\labuser\AppData\Local\Enthought\Canopy\User\lib\site-packages\pandas\io\excel.py:626 error", simply ignore it! The program will still be able to run.

In [1]:
import numpy as np
import scipy
from scipy import spatial
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import time as t
from PIL import Image
import sys
import os, errno
import matplotlib.patches as patches
from pandas.tools.plotting import autocorrelation_plot

def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc: # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else: raise

def load_RAAIM(folder, original):
    data = pd.read_csv(r'%s/ROI normalized.txt' %(folder), index_col= 'time(s)', sep='\t')
    print "Loaded 'ROI normalized.txt'"
    
    roi_param = pd.read_csv(r'%s/Parameter List_edit.txt' %(folder), index_col='ROI', sep='\t')
    print "Loaded 'Parameter List_edit.txt'"
    
    im = Image.open(r'%s/rgb.png' %(folder)) #MUST BE RGB
    print "Loaded 'rgb.png'"
    
    del data['Unnamed: 0'] #lc_pro outputs a weird blank column named this everytime. I don't know why, but it does. this line deletes it safely.
    roi_loc, roi_x, roi_y, data = lcpro_param_parse(roi_param, data , original = original)
    print "Configured Data"
    
    events_x, events_y = get_events(data = data, roi_param = roi_param)
    print "LCPro events extracted"
    
    path = folder +"/plots"
    mkdir_p(path)
    print "Made plots folder"
    
    return data, roi_param, im, roi_loc, roi_x, roi_y, events_x, events_y
    
def lcpro_param_parse(roi_param, data , original):
    '''
    This function takes the Dataframe created by opening the 'Parameter List.txt' from LC_Pro. 
    '''
    roi_loc = roi_param[['X', 'Y']]
    roi_loc.drop_duplicates(inplace= True)
    roi_x = roi_loc['X'].tolist()
    roi_y = roi_loc['Y'].tolist()
    new_index = []
    for i in np.arange(len(roi_loc.index)):
        foo = int(roi_loc.index[i])
        new_index.append('Roi'+str(foo))
    roi_loc = DataFrame({'x':roi_x, 'y':roi_y}, index= new_index)
    
    if len(data.columns) != len(new_index) and original == True:
        sys.exit("The number of ROIs in the data file is not equal to the number of ROIs in the parameter file. That doesn't seem right, so I quit the function for you. Make sure you are loading the correct files, please.")
    
    if original == False:
        data = data[roi_loc.index]
    
    truth = (data.columns == roi_loc.index).tolist()
    
    if truth.count(True) != len(data.columns):
        sys.exit("The names on data and roi_loc are not identical. This will surely break everything later, so I shut down the program. Try loading these files again.")
    
    return roi_loc, roi_x, roi_y, data

def get_events(data, roi_param):
    '''
    extract the events from the roi_parameter list. It returns them as a pair of dictionaries.
    '''
    new_index = []
    for i in np.arange(len(roi_param.index)):
        new_index.append('Roi'+str(roi_param.index[i]))
    roi_events = DataFrame(index= new_index)
    roi_events_time = roi_param['Time(s)'].tolist()
    roi_events_amp = roi_param['Amp(F/F0)'].tolist()
    roi_events['Time'] = roi_events_time
    roi_events['Amp'] = roi_events_amp
    
    events_x = {}
    events_y = {}
    for label in data.columns:
        events_x[label] = []
        events_y[label] = []

    for i in np.arange(len(roi_events.index)):
        key = roi_events.index[i]
        events_x[key].append(roi_events.iloc[i,0])
        events_y[key].append(roi_events.iloc[i,1])
        
    return events_x, events_y

def cross_correlation(data, r):
    '''
    This function calculated the cross correlation between each ROI pair and returns a dataframe with NxN+1  values.
    it also includes a column that contains the number of friends each ROI has.
    lastly, it creates a dictionary where the keys are the roi names and names of each roi's friends are stored as a list. by defalut, each roi is at least friends with itself.
    this file can be saved out as a CSV later.
    '''
    
    data_corr = data.corr()
    ls = (data_corr[data_corr>r].count())
    data_corr['Number of Friends'] = ls
    cols = data_corr.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    data_corr = data_corr[cols]
    
    friend_names = {}
    temp_cor = data.corr()
    correlation_summary = DataFrame()
    
    for label, column in temp_cor.iteritems():
        friend_names[label] = column[column>r].index.tolist()
        friends = column[column>r]
        summary = friends.describe()
        correlation_summary[label] = np.round(summary, 3)

    return data_corr, friend_names, correlation_summary

def distance_matrix(roi_x, roi_y , roi_loc , d):
    '''
    This function calculates the euclidean distance between each ROI pair. It returns a dataframe with these distances.
    it also includes a column that contains the number of neighbors each ROI has.
    the files can be saved out as a CSV later.
    It also returns the list of the (x,y) tuple for each roi.
    '''
    loc_tup = []
    for i in np.arange(len(roi_x)):
        loc_tup.append((roi_x[i], roi_y[i]))
    
    roi_dist = DataFrame(data = (scipy.spatial.distance.cdist(loc_tup, loc_tup, 'euclidean')), index = roi_loc.index, columns= roi_loc.index)
    roi_dist_f = roi_dist
    
    neighbor_names = {}
    distance_summary = DataFrame()
    
    for label, column in roi_dist.iteritems():
        neighbor_names[label] = column[column<=d].index.tolist()
        neighbors = column[column<=d]
        summary = neighbors.describe()
        distance_summary[label] = np.round(summary, 3)

    ls = (roi_dist[roi_dist<=d].count())
    roi_dist_f['Number of Neighbors'] = ls
    cols = roi_dist_f.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    roi_dist_f = roi_dist_f[cols]
    
    return roi_dist_f, loc_tup, neighbor_names, distance_summary

def friend_dist(data_corr, roi_dist, friend_names, r, d):
    '''
    calculates distance information about each roi's friends.
    returns a dataframe with Distance Sum, number of friends, and mean distance. mean dist = distance sum/number of fiends
    '''
    roi_friend_dist = DataFrame(index = data_corr.index)
    #roi_friend_dist['Distance sum'] = (roi_dist[data_corr>r].sum())
    roi_friend_dist['Number of Friends'] = (data_corr[data_corr>r].count())
    roi_friend_dist['Mean Dist'] = (roi_dist[data_corr>r].mean())
    roi_friend_dist['Std Dist'] = (roi_dist[data_corr>r].std())
    
    perc_friends_are_neighbors = Series(index = roi_dist.index) #empty series
    for key in friend_names:
        temp_series = roi_dist.loc[key, friend_names[key]]
        count = temp_series[temp_series<=d].count()
        perc_friends_are_neighbors[key] = count
    roi_friend_dist['Number of Friends that are Neighbors'] = perc_friends_are_neighbors
    roi_friend_dist['Percentage of Friends that are Neighbors'] = (roi_friend_dist['Number of Friends that are Neighbors']/roi_friend_dist['Number of Friends'])*100
    
    return roi_friend_dist

def neighbor_corr(roi_dist, data_corr, neighbor_names, r, d):
    '''
    Calculates correlation information about each roi's neighbors.
    returns dataframe with corr of neighbors, corr of neighbors that are friends, percent of neighbors that are friends
    '''
    
    roi_neighbor_corr = DataFrame(index = roi_dist.index)
    roi_neighbor_corr['Number of Neighbors'] = (roi_dist[roi_dist<=d].count())
    roi_neighbor_corr['Mean Correlation'] = (data_corr[roi_dist<=d].mean())
    roi_neighbor_corr['Std Correlation'] = (data_corr[roi_dist<=d].std())
    
    perc_neighbors_are_friends = Series(index = roi_dist.index)
    for key in neighbor_names:
        temp_series = data_corr.loc[key, neighbor_names[key]]
        count = temp_series[temp_series>r].count()
        perc_neighbors_are_friends[key] = count
    roi_neighbor_corr['Number of Neighbors that are Friends'] = perc_neighbors_are_friends
    roi_neighbor_corr['Percentage of Neighbors that are Friends'] = (roi_neighbor_corr['Number of Neighbors that are Friends']/roi_neighbor_corr['Number of Neighbors'])*100
    
    return roi_neighbor_corr

def onpick(event):
    '''
    allows us to click on an roi and print in the notebook it's name.
    '''
    ind = event.ind
    ind = ind[0]
    print('Roi selected =', roi_loc.index[ind])
    
def all_roi_2d(roi_loc , im , s=6):
    '''
    makes a scatter plot of each roi overlayed on an uploaded image. when you click on an roi, it prints the name of the roi
    for some reason, it takes two clicks to start printing. Not sure why
    '''
    fig, ax = plt.subplots()
    col = ax.scatter(roi_loc.x, roi_loc.y, s=s, edgecolor = 'k', linewidth ='1',color = 'y', marker ="o", alpha = 0.75, picker = True)
    fig.canvas.mpl_connect('pick_event', onpick)

    width, height = im.size
    plt.xlim(xmin=0, xmax=width)
    plt.ylim(ymin = 0, ymax = height)
    plt.title('All LC_Pro ROIs')
    plt.imshow(im)
    plt.show()
    
def friend_roi_2d(data_corr , friend_names, roi_loc, r, d, roi, just_friends, s = 6):
    temp_series_list = data_corr[roi].tolist()
    temp_series = data_corr[roi]
    temp_ls = temp_series[friend_names[roi]]

    fig, ax = plt.subplots()
    if just_friends == False:
        plt.scatter(roi_loc.x, roi_loc.y, s = s, label = 'other objects', edgecolor = 'k', linewidth ='1',c= temp_series_list,cmap=plt.cm.spectral , marker ="o", alpha = 1)
        plt.colorbar()
        plt.clim(-1,1)

        
    if just_friends == True:    
        plt.scatter(roi_loc.x, roi_loc.y, s = s, label = 'other objects', edgecolor = 'k', linewidth ='1',color= 'w' , marker ="o", alpha = 0.25)
        plt.scatter(roi_loc.x[friend_names[roi]], roi_loc.y[friend_names[roi]], label = 'Friends', s=s, edgecolor = 'k', linewidth ='1',c =temp_ls, cmap=plt.cm.spectral , marker ="o", alpha = 1)
        plt.colorbar()
        plt.clim(r,1)
        
    plt.scatter(roi_loc.x[roi], roi_loc.y[roi], label = roi, s=(s+25), edgecolor = 'k', marker ="o", alpha = 1, color = 'w')
    neighborhood = patches.Circle(xy = (roi_loc.x[roi], roi_loc.y[roi]), radius=d, fill = False, color= 'w', lw = 1, ls = 'solid')
    ax.add_artist(neighborhood)
    
    width, height = im.size
    plt.xlim(xmin=0, xmax=width)
    plt.ylim(ymin = 0, ymax = height)
    plt.title(roi)
    plt.imshow(im)
    #plt.legend(loc = 'center left')

    #plt.show()
    plt.savefig(r'%s/friend_2d(r=%s)(d=%s).png' %(folder,r,d))
    
def all_roi_line_pdfs(dataframe, num_rois, ymin, ymax, events_x, events_y, folder):
    
    for r in np.arange(num_rois):
        
        plt.figure()
        plt.xlabel('Time (s)')
        plt.ylabel('Intensity')
        plt.title(dataframe.columns[r])
        plt.ylim(ymin = ymin, ymax = ymax)
        plt.xlim(xmin= dataframe.index[0], xmax = dataframe.index[-1])
        plt.plot(dataframe.index, dataframe.ix[:,r], color = [np.random.random(), np.random.random(), np.random.random()])
        plt.plot(events_x[dataframe.columns[r]], events_y[dataframe.columns[r]],  marker = "^", color="y", linestyle= "None")
        
        plt.savefig(r'%s/plots/%s.pdf' %(folder,dataframe.columns[r]))
        plt.close()
        
def roi_line_plot(roi, dataframe, events_x, events_y, ymin, ymax):
    
    plt.figure()
    plt.xlabel('Time (s)')
    plt.ylabel('Intensity')
    plt.title(roi)
    plt.ylim(ymin = ymin, ymax = ymax)
    plt.xlim(xmin= dataframe.index[0], xmax = dataframe.index[-1])
    plt.plot(dataframe.index, dataframe.ix[:,roi], color = 'k')
    plt.plot(events_x[roi], events_y[roi],  marker = "^", color="y", linestyle= "None")

    plt.show()
    
    
def friend_roi_line(data, friend_names, roi, folder):
    plt.figure()
    plt.plot(data.index, data[friend_names[roi]], color = '#FF6600')
    plt.plot(data.index, data[roi], color = 'k',)
    plt.ylim(ymin = 0, ymax = max(data.max()))
    plt.title(roi)

    #plt.show()
    plt.savefig(r'%s/friend_line(r=%s)(d=%s).png' %(folder,r,d))
    
def friend_roi_line_average(data, friend_names, roi):
    roi_average = data[friend_names[roi]].mean(1)
    roi_std = data[friend_names[roi]].std(1)
    print "the average of the std of each point is", round(np.mean(roi_std),4), "for", roi, "and its friends."

    plt.figure()
    plt.plot(data[friend_names[roi]].mean(1))
    plt.ylim(ymin = 0, ymax = max(data.max()))
    plt.title('Average of '+roi+' and its friends')
    plt.show()

def boxandwhiskers(roi, flavor,data_corr, roi_dist, r, d):
    '''
    Returns the box and whiskers plot of a named ROI's friends or neighbors.
    
    '''
    if flavor == 'corr':
        temp_series = data_corr[roi]
        temp_series = temp_series[temp_series>r]
        plt.boxplot(temp_series)
        plt.ylim(ymin = r*0.85, ymax = 1.1)
        plt.xticks(np.arange(1),roi)
        plt.title(roi)
        plt.text(0.55,r,temp_series.describe())
        plt.ylabel('Correlation (r)')
        plt.show()
        
    
    elif flavor == 'dist':
        temp_series = roi_dist[roi]
        temp_series = temp_series[temp_series<=d]
        plt.boxplot(temp_series)
        plt.ylim(ymin = -3, ymax = d*1.25)
        plt.xticks(np.arange(1),roi)
        plt.title(roi)
        plt.text(0.55,d*0.5,temp_series.describe())
        plt.ylabel('Distance (pixels)')
        plt.show()
        
        
    else:
        print "ERROR: flavor must be set to either 'corr' or 'dist'."        
        
print "RAAIM Ready for Analysis"

ImportError: No module named tools.plotting

# I. Begin User Input

This whole notebook is especially formated for LC_pro outputs. This means that the load functions and wrappers expect that certain columns are in exactly the right locations and labels are formatted a certain way. This author does not expect other types of files to be handled properly (as in: will probably fail). 

## ------------------------------- An Important note about saving!  -------------------------------

This notebook is brutal and will mercilessly **save over exsisting files of the same name without warning!**

You can manually change the name of the saved file one way: 

Change part of the text portion of the VARIBLE.to_csv(r'%s/**OBJECT**.csv' %(folder)). you can change **OBJECT** to whatever you want. go nuts. 

Remember, only you can keep all your files organized and named properly. I'm just here to help. Use a naming convention that is simple and easy to understand/remember. You can totally move ALL of these files to anywhere once you've run this analysis. It won't break stuff.

## ------------------------------------------------------------------------------------------------------

## A. Declare Your Folder

Change the following string below to the location of the folder where your data is. You will also save all of your data in this folder. Must be full path name and not end in a /.

**Example:**

"/Users/abigaildobyns/Desktop/RAAIM"

In [2]:
folder = "C:\Users\labuser\Desktop\Scott 2014\Lab Data\AN_Tiff\AN_Control_2012_03_12__14_19_30.tif Output"
data, roi_param, im, roi_loc, roi_x, roi_y, events_x, events_y = load_RAAIM(folder=folder, original=False)
#roi = 'Roi30'

r = .8 #set r value, must be -1<= r <=1
d = 100 # set the maximum distance from an ROI that is the threshold for neighborhood
#data_corr, friend_names, correlation_summary = cross_correlation(data, r)
#data_corr.to_csv(r'%s/xcor(r=%s)(d=%s).csv' %(folder,r,d))
#correlation_summary.to_csv(r'%s/xcor_summary(r=%s)(d=%s).csv'%(folder,r,d))
#roi_dist, loc_tup, neighbor_names, distance_summary = distance_matrix(roi_x, roi_y , roi_loc , d)
#roi_dist.to_csv(r'%s/dist_matrix(r=%s)(d=%s).csv' %(folder,r,d))
#distance_summary.to_csv(r'%s/dist_summary(r=%s)(d=%s).csv' %(folder,r,d))
#friend_distances = friend_dist(data_corr, roi_dist, friend_names, r, d)
#friend_dist_sum = friend_distances.describe()
#friend_distances.to_csv(r'%s/friend_distances(r=%s)(d=%s).csv' %(folder,r,d))
#friend_dist_sum.to_csv(r'%s/friend_distances_summary(r=%s)(d=%s).csv' %(folder,r,d))
#neighbor_correlation = neighbor_corr(roi_dist, data_corr, neighbor_names, r, d)
#neighbor_corr_sum = neighbor_correlation.describe()
#neighbor_correlation.to_csv(r'%s/neighbor_correlation(r=%s)(d=%s).csv' %(folder,r,d))
#neighbor_corr_sum.to_csv(r'%s/neighbor_correlation_summary(r=%s)(d=%s).csv' %(folder,r,d))

#friend_roi_line(data, friend_names, roi = roi, folder = folder)
#fig1.savefig(r'%s/friend_line.png' %(folder))
#friend_roi_2d(data_corr , friend_names, roi_loc, r, d, roi = roi, just_friends = True, s = 100)
#fig2.savefig(r'%s/friend_2d.png' %(folder))

#friend_roi_line.to_png(r'%s/line_plot_with_friends.png' %(folder))

all_roi_line_pdfs(data, num_rois = len(data.columns), ymin = 0, ymax=max(data.max()), events_x = events_x, events_y = events_y, folder = folder)

#friend_dist_sum[['Number of Friends', 'Mean Dist', 'Percentage of Friends that are Neighbors']]

#neighbor_corr_sum[['Number of Neighbors', 'Percentage of Neighbors that are Friends']]

NameError: name 'load_RAAIM' is not defined

C:\Users\labuser\AppData\Local\Enthought\Canopy\User\lib\site-packages\pandas\util\decorators.py:60: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  return func(*args, **kwargs)


## B. Load

Use the following block below to load your data. There should be three files in this folder:

1. 'ROI normalized.txt'

2. 'Parameter List_edit.txt'

3. 'rbg.png'

There should be no headers (other than column names) in the text files. the text files can be either .txt or .csv. if you have problems loading them, make sure that the data is not corrupted, it is in the correct file type, the delimiter is correct, and the file path is correct. 

the image still can be just a random frame or the averaged or compressed stack. it must be convered to RGB and saved as .png.

###### if you get a *SettingWithCopyWarning*,  **Don't worry, that's normal**. I know that you are setting with a copy. It's on purpose

In [3]:
data, roi_param, im, roi_loc, roi_x, roi_y, events_x, events_y = load_RAAIM(folder=folder, original=False)

NameError: name 'load_RAAIM' is not defined


Made plots folder


###### Use the two cells below to check that the data and roi_loc dataframes look as expected.

In [4]:
data.head()

NameError: name 'data' is not defined

In [5]:
roi_loc.head()

NameError: name 'roi_loc' is not defined

# II. Analysis

## A. Cross Correlation Matrix (friends)

Set your r value to your threshold for significant cross correlation. This value will be used is all later calculation. If you change it, you must rerun ALL cells following to update the change.

In [6]:
r = -0.99 #set r value, must be -1<= r <=1

In [7]:
data_corr, friend_names, correlation_summary = cross_correlation(data, r)
data_corr.head()

NameError: name 'cross_correlation' is not defined

### - Cross Correlation Matrix Summary

In [8]:
correlation_summary

NameError: name 'correlation_summary' is not defined

#### Like what you see? save the cross correlation file with the below code

In [9]:
data_corr.to_csv(r'%s/xcor.csv' %(folder))
correlation_summary.to_csv(r'%s/xcor_summary.csv'%(folder))

NameError: name 'data_corr' is not defined

## B. Distance Matrix (neighbors)

Set your d value to your threshold for significant distance. This value will be used is all later calculation. If you change it, you must rerun ALL cells following to update the change.

In [13]:
d = 100 # set the maximum distance from an ROI that is the threshold for neighborhood

In [14]:
roi_dist, loc_tup, neighbor_names, distance_summary = distance_matrix(roi_x, roi_y , roi_loc , d)
roi_dist.head()

,Number of Neighbors,Roi17,Roi18,Roi22,Roi34,Roi43,Roi52,Roi53,Roi55,Roi56,...,Roi182,Roi185,Roi186,Roi188,Roi189,Roi193,Roi194,Roi195,Roi197,Roi198
Roi17,13,0.000000,32.015621,285.259531,205.380135,294.159821,174.269906,201.556444,203.194488,156.108936,...,348.323126,351.159508,227.727908,315.057138,41.484937,118.330047,343.633526,370.623798,324.376325,185.396872
Roi18,17,32.015621,0.000000,253.428491,194.139125,262.345192,142.593829,187.856328,171.572142,124.149104,...,317.833290,319.390044,198.295234,287.252502,9.486833,86.539009,317.907219,344.280409,295.893562,172.513768
Roi22,22,285.259531,253.428491,0.000000,252.800316,9.000000,111.198022,235.601358,82.710338,129.556937,...,87.863531,66.121101,90.426766,120.415946,244.106534,171.011696,167.836230,176.082367,112.893755,237.438413
Roi34,21,205.380135,194.139125,252.800316,0.000000,258.311827,185.064853,17.204651,194.794764,191.387042,...,261.266148,299.933326,162.409975,195.017948,192.353841,200.900473,195.920902,224.590739,210.639502,23.345235
Roi43,22,294.159821,262.345192,9.000000,258.311827,0.000000,120.033329,241.132744,91.443972,138.520757,...,81.320354,57.140179,96.260064,118.949569,253.031619,180.011111,165.921668,172.754161,110.353976,243.446093


### - Distance Matrix Summary

In [15]:
distance_summary

,Roi17,Roi18,Roi22,Roi34,Roi43,Roi52,Roi53,Roi55,Roi56,Roi58,...,Roi182,Roi185,Roi186,Roi188,Roi189,Roi193,Roi194,Roi195,Roi197,Roi198
count,13.000,17.000,22.000,21.000,22.000,26.000,22.000,31.000,27.000,25.000,...,23.000,13.000,31.000,25.000,20.000,26.000,17.000,13.000,25.000,21.000
mean,44.686,51.962,61.068,50.115,61.846,58.274,48.582,66.785,58.053,53.566,...,66.221,56.707,64.100,55.345,58.195,54.554,62.564,64.333,57.137,51.260
std,23.574,29.564,29.541,26.780,29.163,24.159,26.517,24.102,29.798,29.148,...,26.687,31.170,27.843,26.830,31.457,27.493,25.980,28.671,27.270,30.112
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,35.805,32.016,47.576,27.295,52.335,43.607,25.237,52.641,39.308,34.059,...,59.802,45.100,55.610,34.132,37.650,36.141,47.760,53.907,46.690,23.345
50%,41.485,52.839,66.784,49.041,67.208,65.181,51.391,70.093,50.606,45.277,...,70.661,62.097,67.417,60.008,64.815,55.637,65.803,64.140,59.808,57.689
75%,56.886,77.162,85.182,65.863,84.848,71.244,69.787,87.842,84.592,80.324,...,83.766,78.058,87.833,77.524,82.889,77.717,75.604,85.604,74.947,70.831
max,79.158,91.706,99.298,98.489,96.260,99.690,87.281,97.046,100.000,92.892,...,97.801,94.048,98.793,93.477,98.270,94.530,99.725,99.730,98.793,98.656


#### Like what you see? save the distance matrix file with the below code

In [16]:
roi_dist.to_csv(r'%s/dist_matrix.csv' %(folder))
distance_summary.to_csv(r'%s/dist_summary.csv' %(folder))

## C. Friends in terms of Neighbors

the following function returns a table of the following values:

**Distance Sum**: the sum of the distances from each roi to its friends

**Friend Number**: the number of friends that each roi has (not including itself)

**Rel Dist**: The cummulative distance / number of friends

In [17]:
friend_distances = friend_dist(data_corr, roi_dist, friend_names, r, d)
friend_distances.head()

,Number of Friends,Mean Dist,Std Dist,Number of Friends that are Neighbors,Percentage of Friends that are Neighbors
Roi17,98,209.580441,94.503793,13,13.265306
Roi18,98,188.549529,88.430727,17,17.346939
Roi22,98,173.154914,85.717880,22,22.448980
Roi34,98,184.960542,88.456092,21,21.428571
Roi43,98,177.662536,88.156975,22,22.448980


### - Friends in terms of Neighbors Summary

In [18]:
friend_dist_sum = friend_distances.describe()
friend_dist_sum

,Number of Friends,Mean Dist,Std Dist,Number of Friends that are Neighbors,Percentage of Friends that are Neighbors
count,98,98.000000,98.000000,98.000000,98.000000
mean,98,183.407613,88.491599,21.224490,21.657643
std,0,37.169674,14.559027,7.248933,7.396870
min,98,134.907600,56.304905,1.000000,1.020408
25%,98,156.615980,78.177676,17.000000,17.346939
50%,98,173.118632,89.073788,22.000000,22.448980
75%,98,204.532521,99.320734,26.000000,26.530612
max,98,310.422528,116.694530,34.000000,34.693878


#### Like what you see? save the friend distances file with the below code

In [19]:
friend_distances.to_csv(r'%s/friend_distances.csv' %(folder))
friend_dist_sum.to_csv(r'%s/friend_distances_summary.csv' %(folder))

## D. Neighbors in terms of Friends

This section has yet to be built. Will talk with Sean Wilson about values that make the most sense. here. perhaps number of neightbors and average r value?

In [20]:
neighbor_correlation = neighbor_corr(roi_dist, data_corr, neighbor_names, r, d)
neighbor_correlation.head()

,Number of Neighbors,Mean Correlation,Std Correlation,Number of Neighbors that are Friends,Percentage of Neighbors that are Friends
Roi17,13,0.301327,0.516659,13,100
Roi18,17,0.254530,0.444006,17,100
Roi22,22,0.743534,0.339476,22,100
Roi34,21,0.195455,0.364404,21,100
Roi43,22,0.751660,0.333884,22,100


### - Neighbors in terms of Friends Summary

In [21]:
neighbor_corr_sum = neighbor_correlation.describe()
neighbor_corr_sum

,Number of Neighbors,Mean Correlation,Std Correlation,Number of Neighbors that are Friends,Percentage of Neighbors that are Friends
count,98.000000,98.000000,97.000000,98.000000,98
mean,21.224490,0.336288,0.405078,21.224490,100
std,7.248933,0.194531,0.087254,7.248933,0
min,1.000000,-0.122374,0.163855,1.000000,100
25%,17.000000,0.217800,0.352300,17.000000,100
50%,22.000000,0.299228,0.397314,22.000000,100
75%,26.000000,0.408480,0.470517,26.000000,100
max,34.000000,1.000000,0.562863,34.000000,100


#### Like what you see? save the neighbor correlation file with the below code

In [22]:
neighbor_correlation.to_csv(r'%s/neighbor_correlation.csv' %(folder))
neighbor_corr_sum.to_csv(r'%s/neighbor_correlation_summary.csv' %(folder))

# III. Resulting Plots

## A. 2d Plots

### - All Roi overlay with name calling

For some reason, you have to click on the first cell twice. but be assured, it is working. set the object size with s. default is s = 6

Save this plot from the pop up graph window.

In [28]:
all_roi_2d(roi_loc , im , s=100)

NameError: name 'roi_loc' is not defined

### - Select Roi and Friends plot

Call a 2d plot with each roi on it. Choose an roi by name (ex: 'Roi35') to see it's relationship to all other rois. The named roi will be in white. All other roi's will be colored by its r value. 

when just_friends = True, only friends who meet the r threshold will be shown and all others will be gray. when just_friends = False, all rois will be colored based on r value.

Save this plot from the pop up graph window.

In [29]:
friend_roi_2d(data_corr , friend_names, roi_loc, r, d, roi = 'Roi110', just_friends = True, s = 100)

NameError: name 'data_corr' is not defined

## B. Line Plots

All line plots are scaled the same: 0 to the maximum value in the entire dataframe.

### - Line Plots for all ROIs

Use all_roi_line_pdfs to plot each ROI as its own pdf. Each plot is randomly assigned a color. You cannot disable this feature.

you can easily consolidate all of these into a single pdf using adobe or another preview program.

In [30]:
all_roi_line_pdfs(data, num_rois = len(data.columns), ymin = 0, ymax=max(data.max()), events_x = events_x, events_y = events_y, folder = folder)

NameError: name 'data' is not defined

### - Line Plot for only one ROI

Use roi_line_plot to call an ROI and get its plot. It includes the overlay of the LCPro events

In [31]:
roi_line_plot(roi = 'Roi89', dataframe = data, events_x = events_x, events_y=events_y, ymin = 0, ymax=max(data.max()))

NameError: name 'data' is not defined

### - Stacked line plot of an ROI and its friends

Call an roi by name (ex: 'Roi35') to get a line plot of it and its friends. Save this plot from the pop up graph window.

In [32]:
friend_roi_line(data, friend_names, roi = 'Roi68')

NameError: name 'data' is not defined

### - Average line plot of an ROI and its friends

Call an roi by name (ex: 'Roi35') to get a line plot of the average of it and its friends. the average standard error for each point is printed below once the plot is closed. Save this plot from the pop up graph window.

In [33]:
friend_roi_line_average(data, friend_names, roi = 'Roi215')

NameError: name 'data' is not defined

### - Autocorrelation Plot

In [34]:
roi = 'Roi68'

autocorrelation_plot(data[roi])
plt.title(roi)
plt.show()

NameError: name 'data' is not defined

## C. Box and Whister Plots

Call an ROI and see the distribution of either its friends' correlations or neighbors' distances.

Choose which metrc to use by changing flavor to either 'dist' or 'corr'

In [35]:
boxandwhiskers(roi = 'Roi98', flavor = 'corr', data_corr = data_corr, roi_dist = roi_dist, r = r, d = d)

NameError: name 'data_corr' is not defined

# End

# ............................................................................................................................................

## [ Run the next cells of code if you want to randomize the dataset ]

In [36]:
import random

def shuffle(x):
    random.shuffle(x)
    return x

for label, column in data.iteritems():
    tempcol = data[label].tolist()
    randcol = shuffle(tempcol)  #shuffle the list of values in the column
    data[label] = randcol #write over the original list with randomized list

NameError: name 'data' is not defined

### Check that the dataset set is indeed randomized! Compare this table to the table under the header "(b) Load"

In [37]:
data.head()

NameError: name 'data' is not defined

### Now, go to analysis and it will run the program with the randomized data. (SKIP I. User Input) 

# ............................................................................................................................................